In [ ]:
import sys
sys.path.append('../')
import folium
import osmnx as ox
import pandas as pd
import graphbuilder_v2
import blocksnet_builder
import geopandas as gpd
from availability_estimation import *

In [ ]:
# Загрузка кварталов города
blocks = blocksnet_builder.fetch(id='R1281563', local_crs=32636)

In [ ]:
# Загрузка УДС
city = ox.geocode_to_gdf('R1281563', by_osmid=True)
graph = graphbuilder_v2.get_graph_from_polygon(city, crs=32636, retain_all=False)

In [ ]:
# Создание центроид для кварталов
blocks['centroid'] = blocks.centroid
blocks = blocks.set_geometry('centroid')

In [ ]:
# Расчет средней доступности
adj_mx = availability_matrix(graph,blocks)
p = find_median(blocks,adj_mx)

In [ ]:
# Чистка от inf и соедиение с геометрией кварталов
p_agg = p[p['to_service'] < 1.7e300].copy()
res = gpd.sjoin(p_agg, blocks, how='left', op='within').groupby('index_right').median(['to_service']).reset_index()
res_df = pd.merge(blocks.reset_index(), res, left_on='index', right_on='index_right', how='left')
res_df = res_df.drop(columns=['index_right'])

In [ ]:
# Визуализация
cmap = plt.cm.RdYlGn_r
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
res_df.set_geometry('geometry').plot(column='to_service', cmap=cmap, legend=True, ax=ax)

In [ ]:
# Визуализация интерактивная
m = folium.Map()
res_df = res_df.set_geometry('geometry')
res_df.explore(
    m=m,
    column='to_service',  # Столбец, который определяет цвет
    cmap='RdYlGn_r',  # Использование той же цветовой карты
    legend=True,  # Включение легенды для цветов
    legend_kwds={'caption': 'To Service'},  # Настройка заголовка легенды
)

In [ ]:
# Only для Москвы
# settlement = gpd.read_file('data/geojson/moscow/settlement.geojson')
# district = gpd.read_file('data/geojson/moscow/district.geojson')
# region = gpd.read_file('data/geojson/moscow/region.geojson')

# visualize_availability(res_df, settlement, title='Связанность, мин')
# visualize_availability(res_df, district, title='Связанность, мин')
# visualize_availability(res_df, region, title='Связанность, мин')